In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # Load .env file
openai.api_key = os.getenv("OPENAI_API_KEY")



In [3]:
# Define Helper Function
client = openai.OpenAI()
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content


In [4]:
# Product List and Info
products={
    "Laptops": [
    {
        "name": "Dell XPS 13",
        "brand": "Dell",
        "model_number": "XPS13-9310",
        "warranty": "1 year",
        "rating": 4.7,
        "features": ["13.4-inch display", "16GB RAM", "512GB SSD", "Intel Core i7 processor"],
        "description": "A premium ultrabook with a sleek design and high performance.",
        "price": 1299.99
    },
    {
        "name": "MacBook Air",
        "brand": "Apple",
        "model_number": "MBA-M2",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["13.6-inch Retina display", "8GB RAM", "256GB SSD", "Apple M2 chip"],
        "description": "A lightweight and powerful laptop with long battery life.",
        "price": 1199.99
    }
],
"Phones": [
    {
        "name": "iPhone 15",
        "brand": "Apple",
        "model_number": "iP15",
        "warranty": "1 year",
        "rating": 4.9,
        "features": ["6.1-inch OLED display", "128GB storage", "48MP main camera", "5G"],
        "description": "Apple's latest iPhone with an improved camera and faster performance.",
        "price": 999.99
    },
    {
        "name": "Samsung Galaxy S24",
        "brand": "Samsung",
        "model_number": "SGS24",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["6.2-inch AMOLED display", "256GB storage", "50MP camera", "5G"],
        "description": "A flagship Samsung phone with a beautiful display and excellent camera.",
        "price": 899.99
    }
],
"TVs": [
    {
        "name": "LG OLED C3",
        "brand": "LG",
        "model_number": "OLED55C3",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["55-inch OLED display", "4K resolution", "HDR", "Smart TV"],
        "description": "A top-tier OLED TV with perfect blacks and stunning colors.",
        "price": 1799.99
    },
    {
        "name": "Samsung QLED Q80",
        "brand": "Samsung",
        "model_number": "QN55Q80",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["55-inch QLED display", "4K resolution", "HDR", "Smart TV"],
        "description": "A vibrant QLED TV with excellent brightness and contrast.",
        "price": 1299.99
    }
],
"Desktops": [
    {
        "name": "HP Pavilion Desktop",
        "brand": "HP",
        "model_number": "TP01-2000",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Intel Core i5", "8GB RAM", "512GB SSD", "Windows 11"],
        "description": "A reliable desktop for home and office use.",
        "price": 699.99
    },
    {
        "name": "Apple iMac 24\"",
        "brand": "Apple",
        "model_number": "IMAC24M1",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["24-inch Retina display", "8GB RAM", "256GB SSD", "Apple M1 chip"],
        "description": "A sleek all-in-one desktop with a stunning display and fast performance.",
        "price": 1499.99
    }
]
}

In [5]:
# Define functions to get product by name and category
def get_product_by_name(name: str):
    target = name.strip().lower()           # convert user input into standard lowercase without space version
    for plist in products.values():          # each value is a list
        for p in plist:                      # iterate product dicts
            if p.get("name", "").strip().lower() == target:
                return p
    return None

def get_products_by_category(category: str):
    target = category.strip().lower()
    for key in products.keys():
        if key.lower() == target:
            return products.get(key, [])
    return []


In [6]:
# Classify user queries
import json

def handle_user_message(user_message: str) -> str:
    # 1) classify
    classification_prompt = (
        "Classify the user's query into one of the following: "
        "'Product Inquiry' or 'Support Request'. "
        "If it's a Product Inquiry, extract the product name if mentioned; "
        "optionally extract a category ('Laptops','Phones','TVs','Desktops'). "
        "Return ONLY JSON with keys: type, product (optional), category (optional)."
    )
    messages = [
        {"role": "system", "content": classification_prompt},
        {"role": "user", "content": user_message}
    ]
    classification_result = get_completion_from_messages(messages, temperature=0)
    try:
        cls = json.loads(classification_result)
    except Exception:
        return "Sorry, I couldn’t understand the request. Could you rephrase it?"

    final_reply = None

    # helper to keep product context short
    def summarize_product(p):
        feat = ", ".join(p.get("features", [])[:4])
        return (
            f"- name: {p.get('name')}\n"
            f"  brand: {p.get('brand')}\n"
            f"  model: {p.get('model_number')}\n"
            f"  price: {p.get('price')}\n"
            f"  warranty: {p.get('warranty')}\n"
            f"  rating: {p.get('rating')}\n"
            f"  features: {feat}\n"
            f"  description: {p.get('description')}"
        )

    # 2) branch
    if cls.get("type") == "Product Inquiry":
        product_name = cls.get("product")
        category_name = cls.get("category")
        context_blocks = []

        if product_name:
            p = get_product_by_name(product_name)
            if p:
                context_blocks.append(summarize_product(p))
            else:
                return f'I couldn’t find “{product_name}”. Could you check the name or tell me the category?'
        elif category_name:
            plist = get_products_by_category(category_name)
            if not plist:
                return f'I don’t have products listed under “{category_name}”. Want to try another category?'
            for p in plist[:3]:  # keep context small
                context_blocks.append(summarize_product(p))
        else:
            return "Is there a specific product or category you’d like to ask about?"

        context_text = "\n\n".join(context_blocks)
        ans_messages = [
            {"role": "system",
             "content": "You are a friendly, concise product assistant. "
                        "Use ONLY the provided data. If some info is missing, say so briefly. "
                        "End with one relevant follow-up question."},
            {"role": "assistant", "content": f"Relevant product data:\n{context_text}"},
            {"role": "user", "content": user_message}
        ]
        final_reply = get_completion_from_messages(ans_messages, temperature=0.2, max_tokens=350)

    elif cls.get("type") == "Support Request":
        # optional: sub-classify
        sub_prompt = (
            "Classify the support request (delimited by ####) into:\n"
            "Billing: [Unsubscribe or upgrade, Add a payment method, Explanation for charge, Dispute a charge]\n"
            "Technical Support: [General troubleshooting, Device compatibility, Software updates]\n"
            "Account Management: [Password reset, Update personal information, Close account, Account security]\n"
            "General Inquiry: [Product information, Pricing, Feedback, Speak to a human]\n\n"
            "Output ONLY JSON with keys 'category' and 'subcategory'."
        )
        sub_messages = [
            {"role": "system", "content": sub_prompt},
            {"role": "user", "content": f"####{user_message}####"}
        ]
        sub_raw = get_completion_from_messages(sub_messages, temperature=0, max_tokens=80)
        try:
            sub = json.loads(sub_raw)
        except Exception:
            sub = {"category": "General Inquiry", "subcategory": "Speak to a human"}

        support_messages = [
            {"role": "system",
             "content": "You are a helpful, empathetic customer support agent. "
                        "Be concise, provide clear next steps, and ask exactly one clarifying question if helpful."},
            {"role": "assistant", "content": f"Ticket routing: {json.dumps(sub)}"},
            {"role": "user", "content": user_message}
        ]
        final_reply = get_completion_from_messages(support_messages, temperature=0.2, max_tokens=300)

    else:
        final_reply = "Sorry, I didn’t quite catch that. Ask about a product or describe your support issue."

    return final_reply


In [ ]:
# Build a chat loop so the chatbot can continue to interact with users
def chat_loop():
    print("Assistant ready. Type 'quit' to exit.")
    while True:
        try:
            user_msg = input("You: ").strip()
            if not user_msg:
                continue
            if user_msg.lower() in {"quit", "exit", "q"}:
                print("Assistant: Bye! 👋")
                break
            reply = handle_user_message(user_msg)
            print(f"Assistant: {reply}\n")
        except KeyboardInterrupt:
            print("\nAssistant: Interrupted. Bye! 👋")
            break
        except Exception as e:
            print(f"Assistant (error): {e}")

chat_loop()


Assistant ready. Type 'quit' to exit.


You:  Can you tell me more about your product HP Pavilion Desktop please?


Assistant: The HP Pavilion Desktop (model TP01-2000) is a reliable desktop for home and office use. It features an Intel Core i5 processor, 8GB of RAM, a 512GB SSD, and comes with Windows 11 pre-installed. It is priced at $699.99 and comes with a 1-year warranty. Customers have rated it 4.5 stars. Is there anything specific you would like to know about this product?



You:  What is the storage of Samsung Galaxy S24?


Assistant: The Samsung Galaxy S24 has 256GB of storage. Is there anything else you would like to know about the phone?



You:  Is LG OLED C3 35 inch?


Assistant: The LG OLED C3 model is actually a 55-inch TV. If you have any other questions or need more information, feel free to ask!

